# Time Series Analysis

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import statsmodels.api as sm
from datetime import datetime, timedelta
from scipy import stats

In [3]:
cwd = os.getcwd()
date_format = "%Y-%m-%d"
TORNADO_PATH = os.path.join(cwd, "..", "data/tornadoes/US_tornadoes_2000_2022.csv")
TORNADO_SHAPEFILE = os.path.join(cwd, "..", "data/geographic/US_tornadoes_1950_2022/US_tornadoes_1950_2022.shp")
COUNTIES_PATH = os.path.join(cwd, "..", "data/counties/2018_counties.csv")
COUNTIES_SHAPEFILE = os.path.join(cwd, "..", "data/geographic/cb_2018_us_county_5m/cb_2018_us_county_5m.shp")
INTERSECTION_PATH = os.path.join(cwd, "..", "data/tornado_county_intersections.csv")
PERCENT_PATH = os.path.join(cwd, "..", "data/tornadoes_and_counties_new.csv")
RESULTS_PATH = os.path.join(cwd, "..", "data/results_2000s.csv")

In [10]:
def extract_mean_dev(data_vote, county_id):
    data_county = data_vote[data_vote["fips"]==county_id]
    votes = np.array([data_county["2004"][0], data_county["2008"][0], data_county["2012"][0], data_county["2016"][0], data_county["2020"][0]])
    mean = np.mean(votes)
    std_dev = np.std(votes)
    return (mean,std_dev)

def extractMonths(data_tornado, county_id, startDate, endDate, duration):
    output = []
    currentDate = startDate
    data_county = data_tornado[data_tornado["GEOID"] == county_id]
    
    while currentDate<= endDate:
        data_filtered = data_county[datetime.strftime(data_county["date"],date_format) >= startDate]
        data_filtered = data_filtered[datetime.strftime(data_filtered["date"],date_format) < startDate+ duration]
        tornadoCount_nonzero = len(data_filtered[data_filtered["loss"]>0])
        costAgg = sum(data_filtered["loss"])
        casualties = sum(data_filtered["fat"])+sum(data_filtered["inj"])
        
        output.append([tornadoCount_nonzero,costAgg,casualties])
        currentDate+=duration
    
    return np.array(output)


        
        
    
    

In [13]:
def runTimeSeries():
    data_tornado = pd.read_csv(PERCENT_PATH)
    data_vote = pd.read_csv(RESULTS_PATH)
    
    counties = set(data_vote["fips"])
    duration = timedelta(weeks = 4)
    startTime = datetime.strptime("2000-01-01",date_format)
    endTime = datetime.strptime("2020-12-30",date_format)
    for county_id in counties:
        countyData = extractMonths(data_tornado,county_id, startTime,endTime,duration)
        print(countyData)
        break
    
    # print(startTime)
    
runTimeSeries()

TypeError: descriptor 'strftime' for 'datetime.date' objects doesn't apply to a 'Series' object